<b>Importing necessary packages</b>

In [1]:
import nltk,time,random
from nltk.tag import map_tag
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

<b>Storing Data from NLTK corpus locally into a variable</b>

In [2]:
tagged_dataset = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

* tagged_dataset - Dataset that is used for training and validating the model.

<b>Printing first two elements of the dataset to get feel for how the data is stored</b>

In [3]:
tagged_dataset[:2]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')]]

<b>Printing total set of POS tags present in the dataset</b><br>
Note:- Universal tagset has 12 POS tags.

In [4]:
set([Tuple[1] for sent in tagged_dataset for Tuple in sent])

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

<b>Printing count of total unique word present in the dataset</b>

In [5]:
len(set([Tuple[0] for sent in tagged_dataset for Tuple in sent]))

12408

<b>Segregating Dataset into test and validation set.</b><br>
Note:- Testing is done on data seperately given in the assignment.

In [6]:
train_set, valid_set = train_test_split(tagged_dataset,test_size=0.05,random_state=20)

print(len(train_set))
print(len(valid_set))

3718
196


* train_set - Data set that is used for training.
* valid_set - Data set that is used for validation.

<b>Converting training data into list of tuples</b>

In [7]:
train_tagged_words = [Tuple for sent in train_set for Tuple in sent]

* train_tagged_words - list of (word,POS tag) tuple , obtained from train data set.

<b>getting total set of tags</b>

In [8]:
Tagset = set([Tuple[1] for Tuple in train_tagged_words])

* Tagset - set of POS tags present in train Data set.

<b>Converting validation dataset into list of tuples and words</b><br>
Note:- Below code is used to select n random sentence from the validation set.By passing 196 for n,I'm selecting all the sentence in validation set in a random order.

In [9]:
random.seed(20)
# choose random 5 sents
rndom_num = [random.randint(0,len(valid_set)-1) for x in range(196)]
# list of sents
select_valid_set = [valid_set[i] for i in rndom_num]
# list of tagged words
select_valid_tup = [tup for sent in select_valid_set for tup in sent]
select_valid_words = [tup[0] for sent in select_valid_set for tup in sent]

* rndom_num - list of n random numbers in validation range
* select_valid_set - random sentences chosen from validation set through random numbers present "rndom_num" variable
* select_valid_tup - list of (word,POS tag) tuple present in validation set.
* select_valid_words - list of words present in validation set.

<b>Test dataset hardcoded into the notebook.</b>

In [10]:
test_set=[['Android','is','a','mobile','operating','system','developed','by','Google','.'],
['Android','has','been','the','best-selling','OS','worldwide','on','smartphones','since','2011','and','on','tablets','since','2013','.'],
['Google','and','Twitter','made','a','deal','in','2015','that','gave','Google','access','to','Twitter\'s','firehose','.'],
['Twitter','is','an','online','news','and','social','networking','service','on','which','users','post','and','interact','with','messages','known','as','tweets','.'],
['Before','entering','politics,','Donald','Trump','was','a','domineering','businessman','and','a','television','personality','.'],
['The','2018','FIFA','World','Cup','is','the','21st','FIFA','World','Cup,','an','international','football','tournament','contested','once','every','four','years','.'],
['This','is','the','first','World','Cup','to','be','held','in','Eastern','Europe','and','the','11th','time','that','it','has','been','held','in','Europe','.'],
['Show','me','the','cheapest','round','trips','from','Dallas','to','Atlanta','.'],
['I','would','like','to','see','flights','from','Denver','to','Philadelphia','.'],
['Show','me','the','price','of','the','flights','leaving','Atlanta','at','about','3','in','the','afternoon','and','arriving','in','San','Francisco','.'],
['NASA','invited','social','media','users','to','experience','the','launch','of','ICESAT-2','Satellite','.']]

* test_set - Data set used for testing

## Base functions necessary to build HMM based POS taggers

<b>Function that is used to calculate transition probability from tag "t1" to "t2"</b>

In [11]:
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    #Seperating list of tags in training dataset in order as they appear in sentences.
    train_tag_list = [pair[1] for pair in train_bag]
    #Getting total count of tag that is t1
    count_t1 = len([t for t in train_tag_list if t==t1])
    count_t1_t2 = 0
    #Getting count of tag t1 that is followed by t2
    for index in range(len(train_tag_list)-1):
        if train_tag_list[index]==t1 and train_tag_list[index+1] == t2:
            count_t1_t2 += 1
    return (count_t1_t2, count_t1)

* t2_given_t1() - Name of function that outputs values which is used to calculate transition probability.
* train_tag_list - list of POS tag that is present in training dataset in order as they appear.
* count_t1 - count of POS tag t1 present in the dataset.
* count_t1_t2 - count of POS tags t1 followed by t2.

<b>Function that is used to calculate probability of getting a word given a tag</b><br>
Note:- It is also known as emission probability.

In [12]:
def word_given_tag(word, tag, train_bag = train_tagged_words):
    #getting list of tuples with POS tag that is passed in argument
    tup_list_gn_tag = [pair for pair in train_bag if pair[1]==tag]
    #count of tuples in the obtained list
    tup_count_gn_tag = len(tup_list_gn_tag)
    #getting list of tuples where word and POS tag are as passed in argument
    wd_list_gn_wd = [pair[0] for pair in tup_list_gn_tag if pair[0]==word]
    #count of tuples obtained in the above list
    wd_count_gn_wd = len(wd_list_gn_wd)
    
    return (wd_count_gn_wd, tup_count_gn_tag)

* word_given_tag() - Function that outputs parameters which is used to calculate emission probability.
* tup_list_gn_tag - list of tuples whose POS tag is as passed in the function argument
* tup_count_gn_tag - count of tuples present in tup_list_gn_tag
* wd_list_gn_wd - list of words whose word is as passed in function argument in taglist
* wd_count_gn_wd - count of tuple present in wd_list_gn_wd

<b>Generating a matrix that map all transitions from one tag to another tag</b>

In [13]:
tags_trans_matrix = np.zeros((len(Tagset), len(Tagset)), dtype='float32')
#Populating matrix with transition probability of moving from one POS tag to another POS tag
for i, t1 in enumerate(list(Tagset)):
    for j, t2 in enumerate(list(Tagset)): 
        #transition probability is calculated as (count of tag t1 followed by t2) / (count of tag t1)
        tags_trans_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

tags_trans_matrix

array([[9.36009362e-02, 1.74871743e-01, 5.87705895e-02, 2.72702724e-02,
        2.20412210e-01, 8.82008821e-02, 6.62406608e-02, 5.22005223e-02,
        2.52002524e-03, 9.09909084e-02, 8.09108093e-02, 4.39204387e-02],
       [1.70899034e-02, 5.77686867e-03, 4.81405703e-04, 4.59742434e-02,
        6.37742221e-01, 3.99566740e-02, 3.73089430e-03, 1.26368999e-02,
        2.40702851e-04, 9.26706009e-03, 2.19039600e-02, 2.05199182e-01],
       [3.45149264e-02, 1.20802239e-01, 4.66417900e-04, 7.92910438e-03,
        3.47947747e-01, 1.56716421e-01, 5.78358211e-02, 5.41044772e-02,
        5.13059692e-03, 5.45708947e-02, 4.10447754e-02, 1.18936568e-01],
       [1.62231073e-01, 5.52988052e-02, 1.03585655e-02, 7.34661371e-02,
        6.15139455e-02, 2.05896407e-01, 5.54581657e-02, 2.59760953e-02,
        1.83266938e-01, 1.46454185e-01, 2.86852592e-03, 1.72111560e-02],
       [2.39861414e-01, 1.32749816e-02, 4.20131274e-02, 2.90299058e-02,
        2.63931423e-01, 1.47410646e-01, 4.81400453e-03, 1.71

* tags_matix - matrix used to store transition probability.

<b>Converting the mapping matrix into a DataFrame</b>

In [14]:
tags_trans_df = pd.DataFrame(tags_trans_matrix, columns = list(Tagset), index=list(Tagset))
tags_trans_df

,.,DET,CONJ,X,NOUN,VERB,PRON,ADV,PRT,ADP,NUM,ADJ
.,0.093601,0.174872,0.058771,0.027270,0.220412,0.088201,0.066241,0.052201,0.002520,0.090991,0.080911,0.043920
DET,0.017090,0.005777,0.000481,0.045974,0.637742,0.039957,0.003731,0.012637,0.000241,0.009267,0.021904,0.205199
CONJ,0.034515,0.120802,0.000466,0.007929,0.347948,0.156716,0.057836,0.054104,0.005131,0.054571,0.041045,0.118937
X,0.162231,0.055299,0.010359,0.073466,0.061514,0.205896,0.055458,0.025976,0.183267,0.146454,0.002869,0.017211
NOUN,0.239861,0.013275,0.042013,0.029030,0.263931,0.147411,0.004814,0.017104,0.044019,0.176951,0.009373,0.012217
VERB,0.034940,0.134103,0.005500,0.217230,0.110009,0.169585,0.036179,0.081035,0.031763,0.091571,0.023241,0.064844
PRON,0.041969,0.009920,0.005341,0.092713,0.211751,0.484166,0.008012,0.032430,0.011064,0.022510,0.007249,0.072873
ADV,0.136243,0.067955,0.007328,0.023318,0.031312,0.346436,0.015323,0.080946,0.014324,0.119254,0.030979,0.126582
PRT,0.043151,0.101667,0.002288,0.014057,0.247793,0.398823,0.018634,0.009807,0.001961,0.020922,0.057862,0.083034
ADP,0.039936,0.323216,0.000852,0.034611,0.323642,0.008094,0.069223,0.013419,0.001278,0.017039,0.062726,0.105964


* tags_trans_df - DataFrame which is created from the matrix tags_trans_matrix

## Plain Viterbi Algorithm

<b>Plain Viterbi function implementation without handling any unknown words.</b>

In [15]:
def Plain_Viterbi(words, train_bag = train_tagged_words):
    pred_tag = []
    Tagset = list(set([pair[1] for pair in train_bag]))
    #Below for loop parses through list of all words sequentially.
    for key, word in enumerate(words):
        resultant_prob_lst = []
        #Below for loop parses through list of available tagset.
        for tag in Tagset:
            #Transition Probability is calculated between two tags using Data Frame generated previously.
            #For the starting word there is no intial tag available.so we assign tag "." which 
            #indicates starting of the sentence.
            if key == 0:
                trans_prob = tags_trans_df.loc['.', tag]
            else:
                trans_prob = tags_trans_df.loc[pred_tag[-1], tag]
                
            # Computing Emission probability
            emission_count,emission_tot=word_given_tag(words[key], tag)
            emission_prob = emission_count/emission_tot
            # State Probability is the product of emission and transition probability.
            resultant_prob = emission_prob * trans_prob
            resultant_prob_lst.append(resultant_prob)    
        max_prob = max(resultant_prob_lst)
        
        #Selecting next tag that has maximum probability of occuring.
        max_prob_tag = Tagset[resultant_prob_lst.index(max_prob)] 
        pred_tag.append(max_prob_tag)
    return list(zip(words, pred_tag))

* Plain_Viterbi() - Function that implements plain viterbi algorithm.
* pred_tag - variable used to store sequence of POS tags for the passed word sequence.
* resultant_prob_lst - variable containing list of probability for next POS tag.
* trans_prob -  variable that stores trasitional probability.
* emission_prob -  variable that stores emission probability.
* emission_count - variable that stores count of word given tag in the training data set.
* emission_tot - variable that stores specified tag count in training data set.
* resultant_prob - resultant probability of a tag for the next word.
* max_prob - value of maximum probability from the list of resultant_prob.
* max_prob_tag - variable that stores tag which has maximum probability for the next word.

<b>Predicting tag for words by plain viterbi algorithm for words in validation set</b>

In [16]:
#Tagging the POS tags for validation set
start_time = time.time()
tagged_word_seq = Plain_Viterbi(select_valid_words)
end_time = time.time()
time_diff = end_time-start_time
print("Time taken in seconds: ", time_diff)

Time taken in seconds:  558.8093910217285


* start_time - starting time of prediction.
* end_time - ending time of prediction.
* time_diff - time taken for the prediction to complete.
* tagged_word_seq - validation word sequence which has been tagged.

<b>Calculating Accuracy of plain viterbi algorithm for words in validation sets</b>

In [17]:
#Getting list of tuples whose actual and predicted value matches
correct_pred_list = [i for i, j in zip(tagged_word_seq, select_valid_tup) if i == j] 
#Accuracy is count of tuple that matches / count of total tuple
accuracy = len(correct_pred_list)/len(tagged_word_seq)
accuracy

0.903539639826123

* correct_pred_list - list of (words,POS tag) tuple that matches with acutual value
* accuracy - accuracy of the model

<b>POS tagging for test set</b>

In [18]:
for i in range(len(test_set)):
    print(str(Plain_Viterbi(test_set[i])))
    print('\n\n')

[('Android', '.'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', '.'), ('.', '.')]



[('Android', '.'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', '.'), ('worldwide', '.'), ('on', 'ADP'), ('smartphones', '.'), ('since', 'ADP'), ('2011', '.'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', '.'), ('.', '.')]



[('Google', '.'), ('and', 'CONJ'), ('Twitter', '.'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', '.'), ('that', 'DET'), ('gave', 'VERB'), ('Google', '.'), ('access', 'NOUN'), ('to', 'PRT'), ("Twitter's", '.'), ('firehose', '.'), ('.', '.')]



[('Twitter', '.'), ('is', 'VERB'), ('an', 'DET'), ('online', '.'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('users', 'NOUN'), ('post', 'NOUN'), ('and', 'C

## Method 1 : Viterbi with Rule based Algorithm to handle unknown words

<b>Creating rule based tagger, along with patterns that is used to calculate the tags</b>

In [19]:
patterns = [
    (r'.*ing$', 'VBG'),              # gerund
    (r'.*ed$', 'VBD'),               # past tense
    (r'.*es$', 'VBZ'),               # 3rd singular present
    (r'.*ould$', 'MD'),              # modals
    (r'.*\'s$', 'NN$'),              # possessive nouns
    (r'.*s$', 'NNS'),                # plural nouns
    (r'^-?[0-9]+(.[0-9]+)?$', 'CD'), # cardinal numbers
    (r'.*', 'NN')                    # nouns
]


#Rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

* pattern - list of pattern to be checked by rule based tagger.
* rule_based_tagger - regextagger object

<b>Viterbi function implementation that handles unknown words by rule based tagging</b><br><br>
Note:- In this algorithm if a word is unknown,instead of tagging a random tag for unkown word ,we use rule based tagging method.Please see the comments inside code for understanding of the algorithm.

In [20]:
def Rule_Viterbi(words, train_bag = train_tagged_words):
    pred_tag = []
    Tagset = list(set([pair[1] for pair in train_bag]))
    #Below for loop parses through list of all words sequentially.
    for key, word in enumerate(words):
        resultant_prob_lst = []
        #Below for loop parses through list of available tagset
        for tag in Tagset:
            #Transition Probability is calculated between two tags.
            #For the starting word there is no intial tag available.so we assign tag "." which indicates starting of the sentence.
            if key == 0:
                trans_prob = tags_trans_df.loc['.', tag]
            else:
                trans_prob = tags_trans_df.loc[pred_tag[-1], tag]
                
            # Computing Emission probability
            emission_count,emission_tot=word_given_tag(words[key], tag)
            emission_prob = emission_count/emission_tot
            # State Probability is the product of emission and transition probability.
            resultant_prob = emission_prob * trans_prob
            resultant_prob_lst.append(resultant_prob)
        
        max_prob = max(resultant_prob_lst)
        #If max_prob=0 , it means word is unknown ,then we use rule based tagger as given in else condition.
        if(max_prob!=0.0):
            #Selecting next tag that has maximum probability of occuring.
            max_prob_tag = Tagset[resultant_prob_lst.index(max_prob)] 
        else:
            #map_tag() is used to map normal POS tagset to universal tagset.
            max_prob_tag=map_tag('en-ptb', 'universal',rule_based_tagger.tag([word])[0][1])
        pred_tag.append(max_prob_tag)
    return list(zip(words, pred_tag))

<b>Predicting tag for words by viterbi algorithm,handling unknown words by rule based tagging</b>

In [21]:
#Tagging the POS tags for validation set
start_time = time.time()
tagged_word_seq = Rule_Viterbi(select_valid_words)
end_time = time.time()
time_diff = end_time-start_time
print("Time taken in seconds: ", time_diff)

Time taken in seconds:  513.1577730178833


<b>Calculating Accuracy of viterbi algorithm,along with rule based tagging for handling unknown words </b>

In [22]:
#Getting list of tuples whose actual and predicted value matches
correct_pred_list = [i for i, j in zip(tagged_word_seq, select_valid_tup) if i == j]
#Accuracy is count of tuple that matches / count of total tuple
accuracy = len(correct_pred_list)/len(tagged_word_seq)
accuracy

0.9503208445456427

In [23]:
for i in range(len(test_set)):
    print(str(Rule_Viterbi(test_set[i])))
    print('\n\n')

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]



[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'VERB'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.')]



[('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ("Twitter's", 'X'), ('firehose', 'NOUN'), ('.', '.')]



[('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('use

## Method 2:- Viterbi with Probabilistic method to handle unkown words

<b>viterbi algorithm where unkown words are handled by probabilistic approach</b>

In [24]:
def Prob_Viterbi(words, train_bag = train_tagged_words):
    pred_tag = []
    Tagset = list(set([pair[1] for pair in train_bag]))
    #Below for loop parses through list of all words sequentially.
    for key, word in enumerate(words):
        resultant_prob_lst = []
        Trans_prob=[]
        #Below for loop parses through list of available tagset
        for tag in Tagset:
            #Transition Probability is calculated between two tags.
            #For the starting word there is no intial tag available.so we assign tag "." which indicates starting of the sentence.
            if key == 0:
                trans_prob = tags_trans_df.loc['.', tag]
            else:
                trans_prob = tags_trans_df.loc[pred_tag[-1], tag]
                
            # Computing Emission probability
            emission_count,emission_tot=word_given_tag(words[key], tag)
            emission_prob = emission_count/emission_tot
            # State Probability is the product of emission and transition probability.
            resultant_prob = emission_prob * trans_prob
            resultant_prob_lst.append(resultant_prob)
            Trans_prob.append(trans_prob)

        max_prob = max(resultant_prob_lst)
        max_prob_tag = Tagset[resultant_prob_lst.index(max_prob)]
        #If emission probability is 0,then we consider only transition probability to decide next POS tag
        if(max_prob==0.0):
            max_prob = max(Trans_prob)
            max_prob_tag = Tagset[Trans_prob.index(max_prob)]
        pred_tag.append(max_prob_tag)
    return list(zip(words, pred_tag))

Trans_prob - variable containing list of probability of next POS tag

<b>Predicting tag for words by viterbi algorithm,handling unknown words by probabilistic tagging</b>

In [25]:
#Tagging the POS tags for validation set
start_time = time.time()
tagged_word_seq = Prob_Viterbi(select_valid_words)
end_time = time.time()
time_diff = end_time-start_time
print("Time taken in seconds: ", time_diff)

Time taken in seconds:  514.9112617969513


<b>Calculating Accuracy of viterbi algorithm,along with probabilistic tagging for unknown words </b>

In [26]:
#Getting list of tuples whose actual and predicted value matches
correct_pred_list = [i for i, j in zip(tagged_word_seq, select_valid_tup) if i == j]
#Accuracy is count of tuple that matches / count of total tuple
accuracy = len(correct_pred_list)/len(tagged_word_seq)
accuracy

0.9389360380873525

In [27]:
for i in range(len(test_set)):
    print(str(Prob_Viterbi(test_set[i])))
    print('\n\n')

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]



[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.')]



[('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NOUN'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'X'), ('access', 'NOUN'), ('to', 'PRT'), ("Twitter's", 'VERB'), ('firehose', 'X'), ('.', '.')]



[('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('use

## Result:-

* <b>Accuracy of plain Viterbi algorithm:- 90.35%</b>
* <b>Accuracy of Viterbi algorithm with rule based method to handle unknown words:- 95.03%</b>
* <b>Accuracy of Viterbi algorithm with probabilistic method to handle unknown words:- 93.89%</b>


<b>Cases where words are correctly tagged for viterbi along with rule based algorithm:-</b>

* Android - Conj to Noun 
* google - Conj to Noun
* os - Conj to Noun 
* worldwide - Conj to Noun 
* 2011 - Conj to Num 
* 2013 - Conj to Num 
* twitter - Conj to Noun 
* online - Conj to Noun 
* tweets - Conj to Noun
* politics - Conj to Noun 
* domineering - Conj to Verb
* fifa - Conj to Noun
* cup - Conj to Noun
* 21st - Conj to Noun
* tournament - Conj to Noun
* contested - Conj to Verb
* arriving - Conj to Verb
* Nasa - Conj to Noun
* icesat-2 - Conj to Noun
* satellite - Conj to Noun 
* invited - Conj to Verb


<b>Cases where words are correctly tagged for viterbi along with probabilistic algorithm:-</b>

* Android - Conj to Noun
* google - Conj to Noun
* os - Conj to Noun
* worldwide - Conj to Noun
* 2013 - Conj to Num
* smartphones - Conj to Noun
* twitter - Conj to Noun
* online - Conj to Noun
* tweets - Conj to Noun
* personality - Conj to Noun
* fifa - Conj to Noun
* cup - Conj to Noun
* tournament - Conj to Noun
* Trips - Conj to Noun
* Nasa - Conj to Noun
* icesat-2 - Conj to Noun
* satellite - Conj to Noun